# Installing all the dependencies 

In [1]:
import numpy as np
import cv2 
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
from pathlib import Path
import glob
import os 

# Adding Mediapipe Holistic 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


# Getting real time videocam

In [3]:


def modifyframe(frame):
    # Example: Convert to grayscale
    return frame


In [4]:
def start():
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)

    # Set camera properties
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)

    if not cap.isOpened():
        print("Camera couldn't be opened.")
        return

    print("Press 'q' to quit.")

    with mp_holistic.Holistic(
        min_detection_confidence=0.5, 
        min_tracking_confidence=0.5,
                 ) as holistic:
        # Defined parameters to the holistic model 
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame.")
                break

            # Apply any preprocessing (e.g., grayscale, filters)
            modified = modifyframe(frame)
            frame = cv2.flip(modified, 1)

            # To improve performance, mark the image as not writeable
            frame.flags.writeable = False
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb_frame)

            # Draw landmark annotation on the image
            frame.flags.writeable = True
            frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

            # Draw face landmarks
            mp_drawing.draw_landmarks(
                frame,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS
            )
            
            # Draw pose landmarks
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )
            
            # Draw left-hand landmarks
            mp_drawing.draw_landmarks(
                frame,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS
            )

            # Draw right-hand landmarks
            mp_drawing.draw_landmarks(
                frame,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS
            )


            # Show the frame
            cv2.imshow("Landmarks", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


In [5]:
start()

Press 'q' to quit.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [6]:

  

print(mp.__version__)

0.10.11


# Landmark Extraction And Data Exploration 

In [46]:

video_dir = Path("Data/INCLUDE/Society/Videos")
landmark_dir = Path("Data/INCLUDE/Society/Landmarks")

#Create same class named folder in Landmark directory 

class_names = []
# Use .iterdir() to loop through the contents of the directory
for item in video_dir.iterdir():
    # Check if the item is a directory before adding its name
    if item.is_dir():
        class_names.append(item.name)

# Loop through your list of names and create a new folder for each
for name in class_names:
    # Construct the full path for the new folder
    new_folder_path = os.path.join(landmark_dir, name)
    
    # Create the directory. exist_ok=True prevents an error if it already exists.
    os.makedirs(new_folder_path, exist_ok=True)

print("✅ Folders created successfully!")


def show():
    #load the first video 
    avi_files = list(video_dir.glob("*.MP4"))
    
    # Check if at least one .avi file was found
    if avi_files:
        first_video_path = avi_files[1]  # This is a Path object
        cap = cv2.VideoCapture(str(first_video_path))  # Convert to string for OpenCV
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 580)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
        if not cap.isOpened():
            print("Error: Could not open video.")
        else:
            print(f"Playing: {first_video_path.name}")
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
    
                cv2.imshow('Video Playback', frame)
    
                # Press 'q' to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()
            
    else:
        print("No .avi files found.")




✅ Folders created successfully!


In [47]:
#show()

## Converting to Landmarks and Saving as .npy files 


In [48]:
#Initialize mediapipe holistic\
#https://github.com/google-ai-edge/mediapipe/blob/master/docs/solutions/holistic.md

holistic = mp_holistic.Holistic(
    static_image_mode = False,
    model_complexity =1,
    smooth_landmarks = True,
    refine_face_landmarks = True,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
)



# Globals to store expected face landmark count
EXPECTED_FACE_LANDMARKS = None

# Helper to extract and concatenate landmarks
def extract_frame_landmarks(results):
    global EXPECTED_FACE_LANDMARKS
    lm = []
    
    # Right hand (21 points × x,y)
    if results.right_hand_landmarks:
        for p in results.right_hand_landmarks.landmark:
            lm.extend([p.x, p.y])
    else:
        lm.extend([0] * 42)
        
    # Left hand3
    if results.left_hand_landmarks:
        for p in results.left_hand_landmarks.landmark:
            lm.extend([p.x, p.y])
    else:
        lm.extend([0] * 42)
        
    # Face (468 points × x,y = 936 values)
    face_lms =  results.face_landmarks.landmark if results.face_landmarks else []
    count = len(face_lms)

    if EXPECTED_FACE_LANDMARKS is None:
        EXPECTED_FACE_LANDMARKS = count or 468

    for i in range(min(count,EXPECTED_FACE_LANDMARKS)):
        p = face_lms[i]
        lm.extend([p.x,p.y])

    missing = EXPECTED_FACE_LANDMARKS - min(count, EXPECTED_FACE_LANDMARKS)
    if missing > 0:
        lm.extend([0.0, 0.0] * missing)

    
    # ---- Final consistency check ----
    total_features = 42 * 2 + EXPECTED_FACE_LANDMARKS * 2
    if len(lm) != total_features:
        raise ValueError(f"Inconsistent landmark length {len(lm)} vs expected {total_features}")

    return np.array(lm, dtype=np.float32)



def process_video(video_dir, landmark_dir,holistic_model):
    for file in os.listdir(video_dir):
        if not file.lower().endswith((".avi", ".mp4",".mov")):
            continue
            
        video_path = os.path.join(video_dir, file)
        
        cap = cv2.VideoCapture(video_path)
        frames_lm = []
    
        print(f"Processing {file}...")
        frame_count = 0
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # Convert BGR to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb_frame)
            
            # Extract and store landmarks
            frame_landmarks = extract_frame_landmarks(results)  # Store in variable
            frames_lm.append(frame_landmarks)                   # Append to list
            print(f"Frame {frame_count}: shape = {frame_landmarks.shape}")  # Now defined!
            
            frame_count += 1
    
        cap.release()

        # Check if all frames have same shape before stacking
        if frames_lm:
            shapes = [arr.shape for arr in frames_lm]
            print(f"All frame shapes: {set(shapes)}")  # Should show only one unique shape
            
            # Stack into [num_frames × num_features]
            sequence = np.stack(frames_lm, axis=0)
            # Save to .npy
            out_file = os.path.join(landmark_dir, file.rsplit(".", 1)[0] + ".npy")
            np.save(out_file, sequence)
            print(f"Saved landmarks to {out_file} with shape {sequence.shape}")
        



W0000 00:00:1758260635.870780   84396 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260635.895682   84396 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260635.896807   84402 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260635.896874   84399 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260635.898035   84400 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260635.907850   84399 inference_feedback_manager.cc:114] Feedback manager 

In [49]:
#find average frames
def average_frames(video_dir,landmark_dir,class_averages):

    # Loop through each class folder and process it
    for class_dir in class_folders:
            class_name = class_dir.name
            
     # --- Calculate and STORE the average frames for THIS class ---
            try:
                file_list = [f for f in os.listdir(landmark_dir_for_class) if f.endswith(".npy")]
                
                if not file_list:
                    print(f"⚠️  No landmark files found for class '{class_name}'. Skipping average calculation.\n")
                    continue
    
                frame_counts = [np.load(os.path.join(landmark_dir_for_class, f)).shape[0] for f in file_list]
    
                if frame_counts:
                    average_frame_count = np.mean(frame_counts)
                    # Store the average in the dictionary
                    class_averages[class_name] = average_frame_count
            except Exception as e:
                print(f"❌ An error occurred while calculating average for '{class_name}': {e}\n")
    
    
    if class_averages:
        # Sort the results alphabetically by class name for a clean report
        for class_name, avg_frames in sorted(class_averages.items()):
            print(f"  ▶️  Class '{class_name}': {avg_frames:.2f} average frames")
    else:
        print("  No class averages were calculated.")
    
    print("\n🎉 All classes have been processed successfully!")

In [50]:
# =====================================================================================
#                         AUTOMATION CELL TO PROCESS ALL CLASSES
# =====================================================================================
from pathlib import Path
import numpy as np
import os

# --- 1. DEFINE YOUR PARENT DIRECTORIES HERE ---
# This is the folder that contains all your class folders (e.g., Big, Small, etc.)
BASE_VIDEO_PATH = video_dir

# This is the folder where the corresponding landmark folders will be created
BASE_LANDMARK_PATH = landmark_dir
# ----------------------------------------------------


# Find all the class subdirectories in the base video path
try:
    class_folders = [d for d in BASE_VIDEO_PATH.iterdir() if d.is_dir()]
    class_names = [d.name for d in class_folders]
    print(f"✅ Found {len(class_names)} classes: {class_names}\n")
except FileNotFoundError:
    print(f"❌ ERROR: The directory '{BASE_VIDEO_PATH}' was not found. Please check the path.")
    class_folders = [] # Ensure the script doesn't crash if path is wrong

# Create a dictionary to store the results for the final summary
class_averages = {}

# Initialize the model here using a 'with' block that wraps the main loop.
with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    refine_face_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as holistic:

    # Loop through each class folder and process it
    for class_dir in class_folders:
        class_name = class_dir.name
        
        print(f"================== Processing Class: {class_name} ==================")
        
        # Define the specific video and landmark directories for the current class
        video_dir_for_class = class_dir
        landmark_dir_for_class = BASE_LANDMARK_PATH / class_name
        
        # Create the landmark directory for the class if it doesn't already exist
        os.makedirs(landmark_dir_for_class, exist_ok=True)
        
        # --- Run the video processing ---
        # This will convert all videos in the class folder to landmark .npy files
        process_video(video_dir_for_class, landmark_dir_for_class, holistic)
        
        # --- Calculate and STORE the average frames for THIS class ---
        try:
            file_list = [f for f in os.listdir(landmark_dir_for_class) if f.endswith(".npy")]
            
            if not file_list:
                print(f"⚠️  No landmark files found for class '{class_name}'. Skipping average calculation.\n")
                continue

            frame_counts = [np.load(os.path.join(landmark_dir_for_class, f)).shape[0] for f in file_list]

            if frame_counts:
                average_frame_count = np.mean(frame_counts)
                # Store the average in the dictionary
                class_averages[class_name] = average_frame_count
                print(f"✅ Finished processing class '{class_name}'.\n")
            else:
                print(f"▶️  No frames to average for class '{class_name}'.\n")

        except Exception as e:
            print(f"❌ An error occurred while calculating average for '{class_name}': {e}\n")


# ===============================================================================
# FINAL SUMMARY REPORT
# ===============================================================================
print("========================================================")
print("                    📊 FINAL SUMMARY                    ")
print("========================================================")

average_frames(video_dir,landmark_dir,class_averages)

✅ Found 7 classes: ['Ball', 'Exercise', 'God', 'Price', 'Science', 'Sign', 'Sport']

================== Processing Class: Ball ==================
Processing MVI_4324.MOV...
Frame 0: shape = (1040,)
Frame 1: shape = (1040,)
Frame 2: shape = (1040,)
Frame 3: shape = (1040,)


W0000 00:00:1758260637.339936   84424 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260637.365259   84424 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260637.366673   84428 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260637.366847   84422 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260637.367246   84424 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758260637.381311   84418 inference_feedback_manager.cc:114] Feedback manager 

Frame 4: shape = (1040,)
Frame 5: shape = (1040,)
Frame 6: shape = (1040,)
Frame 7: shape = (1040,)
Frame 8: shape = (1040,)
Frame 9: shape = (1040,)
Frame 10: shape = (1040,)
Frame 11: shape = (1040,)
Frame 12: shape = (1040,)
Frame 13: shape = (1040,)
Frame 14: shape = (1040,)
Frame 15: shape = (1040,)
Frame 16: shape = (1040,)
Frame 17: shape = (1040,)
Frame 18: shape = (1040,)
Frame 19: shape = (1040,)
Frame 20: shape = (1040,)
Frame 21: shape = (1040,)
Frame 22: shape = (1040,)
Frame 23: shape = (1040,)
Frame 24: shape = (1040,)
Frame 25: shape = (1040,)
Frame 26: shape = (1040,)
Frame 27: shape = (1040,)
Frame 28: shape = (1040,)
Frame 29: shape = (1040,)
Frame 30: shape = (1040,)
Frame 31: shape = (1040,)
Frame 32: shape = (1040,)
Frame 33: shape = (1040,)
Frame 34: shape = (1040,)
Frame 35: shape = (1040,)
Frame 36: shape = (1040,)
Frame 37: shape = (1040,)
Frame 38: shape = (1040,)
Frame 39: shape = (1040,)
Frame 40: shape = (1040,)
Frame 41: shape = (1040,)
Frame 42: shape = 